# Getting started

Once you've chosen your scenario, download the data from [the Iowa website](https://data.iowa.gov/Economy/Iowa-Liquor-Sales/m3tr-qhgy) in csv format. Start by loading the data with pandas. You may need to parse the date columns appropriately.

#### Scenario 2: Market research for new store locations

A liquor store owner in Iowa is looking to expand to new locations and has hired you to investigate the market data for potential new locations. The business owner is interested in the details of the best model you can fit to the data so that his team can evaluate potential locations for a new storefront.

**Goal for Scenario #2:** Your task is to:

* Build models of total sales based on location, price per bottle, total bottles sold. You may find it useful to build models for each county, zip code, or city.
* Provide a table of the best performing stores by location type of your choice (city, county, or zip code) and the predictions of your model(s).
* Based on your models and the table of data, recommend some general locations to the business owner, taking into account model performance. Validate your model's performance and ability to predict future sales using cross-validation.
* _Bonus_: Recommend targets for volume sold and price per bottle!

---

In [14]:
%matplotlib inline

import pandas as pd
import numpy as np
import pprint

In [15]:
## Load the data into a DataFrame
df = pd.read_csv("Iowa_Liquor_sales_sample_10pct.csv")

## Transform the dates if needed, e.g.
# df["Date"] = pd.to_datetime(df["Date"], format="%m-%d-%y")

df.head()

,,,,,,,,,,,,,,,,,,<<<<<<< HEAD
NaN,Date,Store Number,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,Item Number,Item Description,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0.0,2015-11-04,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,55,54436,Mr. Boston Apricot Brandy,750,4.5,6.75,12,81.0,9.0,2.38
1.0,2016-03-02,2614,DAVENPORT,52807,82.0,Scott,1011100.0,BLENDED WHISKIES,395,27605,Tin Cup,750,13.75,20.63,2,41.26,1.5,0.4
2.0,2016-02-11,2106,CEDAR FALLS,50613,7.0,Black Hawk,1011200.0,STRAIGHT BOURBON WHISKIES,65,19067,Jim Beam,1000,12.59,18.89,24,453.36,24.0,6.34
3.0,2016-02-03,2501,AMES,50010,85.0,Story,1071100.0,AMERICAN COCKTAILS,395,59154,1800 Ultimate Margarita,1750,9.5,14.25,6,85.5,10.5,2.77


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 541914 entries, (nan, Date, Store Number, City, Zip Code, County Number, County, Category, Category Name, Vendor Number, Item Number, Item Description, Bottle Volume (ml), State Bottle Cost, State Bottle Retail, Bottles Sold, Sale (Dollars), Volume Sold (Liters)) to (>>>>>>> e16f0559d51cc8e04f1e2f419570debac0262fdf, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan)
Data columns (total 1 columns):
<<<<<<< HEAD    270956 non-null object
dtypes: object(1)
memory usage: 8.3+ MB


### Data Munging

In [13]:
df['Date'] = df['Date'].apply(pd.to_datetime)

KeyError: 'Date'

In [ ]:
df['Bottle Volume (ml)'].hist(bins = 200)

Most common bottle type is 750ml. 1750ml is second most popular.

In [ ]:
# Function to convert money columns from string to float

def conv_money(x):
    x = x.split('.')
    return (float(x[0][1:]) + (float(x[1]) / 100))

In [ ]:
# Converting columns with money value to float

df[['State Bottle Cost', 'State Bottle Retail', "Sale (Dollars)"]] = \
df[['State Bottle Cost', 'State Bottle Retail', "Sale (Dollars)"]].applymap(conv_money)

In [ ]:
df['Item Description'] = df['Item Description'].apply(lambda x: x.strip())

In [ ]:
df.info()

## Cleaning Zipcodes

In [ ]:
## Checking to see if all zipcodes have length of 5

df[df['Zip Code'].apply(lambda x: len(x) != 5)]

In [ ]:
# Since all has 5, change all to int types

df[df['Zip Code'].apply(lambda x: int(x))]

In [ ]:
# Even all is length 5, 1 entry was '712-2'
# Googled this, turned out to be area code, change to 51529

df[df['Zip Code'].apply(lambda x: x == '712-2')]

In [ ]:
# Changed erroneous zipcode

df.loc[df['Zip Code'] == '712-2', 'Zip Code'] = '51529'


In [ ]:
# Changed zipcodes to int type

df['Zip Code'] = df['Zip Code'].apply(lambda x: int(x))

## Cleaning City Values

In [ ]:
list(df['City'].unique())

In [ ]:
# There are some 'City' entries which are in lower case while the majority is in Upper Case. 
# To prevent double counting, all "City" entries are changed to upper case

df['City'] = df['City'].apply(lambda x: x.upper())

## Cleaning NaN County entries

In [ ]:
# Create a new dataframe for rows with NaNs in any cells

df_nan = df[pd.isnull(df).any(axis = 1)]

In [ ]:
df_nan.head()

In [ ]:
# Printing columns that contain "NaN" and count the number of NaN entries per column

tbc_list = []

for col in df.columns:
    x = sum(pd.isnull(df[col]))
    if x != 0:
        tbc_list.append(col)
        print col, x

In [ ]:
# Some counties and county numbers are not filled.
# Since "City" column does not contain NaN and Cities are a subset of counties,
# we try to see if there are similar cities with non-empty county values which we can then copy to the NaN county cells


# Create a set of Cities with NaN for county or county Number

city_nan_list = list(df_nan['City'].unique())

city_nan_list

In [ ]:
# Creating a dictionary with "City" as key and collecting all possible corresponding "County" and "County Name"
# as values of the dictionary

nan_cty_dict = {}

for j in city_nan_list:
    c_set = {x for x in df[df['City']==j]['County'] if x > 0} # x > 0 to skip NaN values
    num_set = {x for x in df[df['City']==j]['County Number'] if x > 0}
    nan_cty_dict[j] = [list(c_set), list(num_set)]

nan_cty_dict

In [ ]:
## Replacing NaN county cells using corresponding city value as key

for cty in nan_cty_dict:
    try:
        df.loc[df['City'] == cty, 'County'] = nan_cty_dict[cty][0][0]
        df.loc[df['City'] == cty, "County Number"] = nan_cty_dict[cty][1][0]
    except:
        pass

In [ ]:
# Recreate a new df to check remaining NaN row
df_nan = df[pd.isnull(df[['County','County Number']]).any(axis = 1)]

In [ ]:
# Collect cities with NaN values
city_nan_list = list(df_nan['City'].unique())

city_nan_list

In [ ]:
nan_cty_dict = {}

for j in city_nan_list:
    c_set = {x for x in df[df['City']==j]['County']} # x > 0 to skip NaN values
    num_set = {x for x in df[df['City']==j]['County Number']}
    nan_cty_dict[j] = [list(c_set), list(num_set)]

nan_cty_dict

In [ ]:
# Updating counties by hand

df.loc[df['City'] == 'TABOR', 'County'] = 'Fremont'
df.loc[df['City'] == 'RUNNELLS', 'County'] = 'Polk'
df.loc[df['City'] == 'SEYMOUR', 'County'] = 'Wayne'

In [ ]:
# Recreate a new df to check remaining NaN row
df_nan = df[pd.isnull(df['County'])]

In [ ]:
len(df_nan)

#### Comments:

Remaining 12 rows with empty counties were filled manually. Without a reference of how county numbers were developed, we left the empty county numbers empty. Analyses based on counties will be performed on 'County' instead of 'County Number'.

# ---------------------------------------------------------------

## Cleaning Empty Category Name using Item Decription

In [ ]:
df_nan = df[pd.isnull(df['Category Name'])]
print len(df_nan)
df_nan

In [ ]:
# Create a list of "Item Description" with NaN in rows
item_nan_list = list(df_nan['Item Description'].unique())

item_nan_list

In [ ]:
item_dict = {}

for j in item_nan_list:
    c_set = {x for x in df[df['Item Description']==j]['Category Name'] if x > 0}
    item_dict[j] = list(c_set)

item_dict

In [ ]:
for drink in item_dict:
    try:
        df.loc[df['Item Description'] == drink, 'Category Name'] = item_dict[drink][0]
    except:
        pass

In [ ]:
df_nan = df[pd.isnull(df['Category Name'])]
print len(df_nan)
df_nan

In [ ]:
df_nan['Item Description'].unique()

In [ ]:
package = ['Jack Daniels TN Honey & Jack Daniels TN Fire 750mls w/2Gls', 'Bombay Sapphire w/Glass',\
           'Evan Williams Egg Nog', 'Pinnacle Candy Cane Trial Pack', 'Wild Turkey American Honey Shot Tap',\
           'Knob Creek w/ Crystal Decanter']

for i in package:
    df.loc[df['Item Description'] == i,'Category Name'] = 'DECANTERS & SPECIALTY PACKAGES'

In [ ]:
df.loc[df['Item Description'] == 'Woodford Reserve MC 1838 White Corn',\
       'Category Name'] = 'CORN WHISKIES'

In [ ]:
lique = ['Rothman & Winter Orchard Peach', 'Giffard Creme de Pamplemousse Rose Pink Grapefruit Liqueur',\
        'Pearl Caramel']

for i in lique:
    df.loc[df['Item Description'] == i,'Category Name'] ='MISC. IMPORTED CORDIALS & LIQUEURS'

In [ ]:
df.loc[df['Item Description'] == 'Glenfiddich 1963 Replica HA',\
       'Category Name'] ='SINGLE MALT SCOTCH'

In [ ]:
df.loc[df['Item Description'] == 'Parkers Heritage Collection Malt Whiskey HA',\
       'Category Name'] = "STRAIGHT BOURBON WHISKIES"

In [ ]:
vod = ['High Stick Vodka', 'Ciroc Ten HA']

for i in vod:
    df.loc[df['Item Description'] == i,'Category Name'] = 'IMPORTED VODKA'

df.loc[df['Item Description'] == 'Absolut w/Ginger Beer','Category Name'] = 'VODKA FLAVORED'

#### Comments:

There are still cells in 'Category' with NaN values, however, since we will only be using "Category Name" for analyses, NaNs in 'Category' were not cleaned.

## Groupby County

In [ ]:
pd.pivot_table(df, index = ['County', 'Store Number'], values = ['Sale (Dollars)'], \
               aggfunc = [np.mean, max, min, np.sum]).\
                sort_values([('sum', "Sale (Dollars)")], ascending = False).head(10)

In [254]:
pd.pivot_table(df, index = ['County', 'Store Number'], values = ['Sale (Dollars)'], \
               aggfunc = [np.mean, max, min, np.sum]).\
                sort_values([('mean', "Sale (Dollars)")], ascending = False).head(10)

mean            max            min  \
                           Sale (Dollars) Sale (Dollars) Sale (Dollars)   
County        Store Number                                                
Dubuque       9018            6435.000000        6435.00        6435.00   
Polk          3814            2234.074820       32184.00         121.56   
Johnson       4677            1782.670108       29327.76          66.93   
              9001            1204.865000       14602.50         110.70   
Polk          3420             936.223395       16200.00          38.40   
Linn          3385             855.724814       15414.00          64.02   
Scott         9002             797.615510        6226.08          91.56   
Polk          5144             787.377406        8279.52          62.28   
Scott         3354             750.236802        7404.60          64.02   
Pottawattamie 3477             736.431774       16542.00          42.00   

                                      sum  
                           Sale (Dollars)  
County        Store Number                 
Dubuque       9018                6435.00  
Polk          3814              310536.40  
Johnson       4677              165788.32  
              9001               28916.76  
Polk          3420              402576.06  
Linn          3385              506589.09  
Scott         9002               39083.16  
Polk          5144              166924.01  
Scott         3354              295593.30  
Pottawattamie 3477              182635.08

## Groupby Zipcode

In [255]:
pd.pivot_table(df, index = ['Zip Code', 'Store Number'], values = ['Sale (Dollars)'], \
               aggfunc = [np.mean, max, min, np.sum]).\
                sort_values([('sum', 'Sale (Dollars)')], ascending = False).head(10)

,,mean,max,min,sum
,,Sale (Dollars),Sale (Dollars),Sale (Dollars),Sale (Dollars)
Zip Code,Store Number,,,,
50320,2633,491.080519,36392.40,4.50,1220335.09
50314,4829,473.544726,13233.60,6.00,1089152.87
52240,2512,302.767702,16542.00,7.26,532265.62
52402,3385,855.724814,15414.00,64.02,506589.09
50311,3420,936.223395,16200.00,38.40,402576.06
52722,3952,344.255640,6616.80,9.50,392451.43
50266,3814,2234.074820,32184.00,121.56,310536.40
52401,3773,253.965306,2538.72,7.05,298663.20


In [256]:
pd.pivot_table(df, index = ['Zip Code'], values = ['Sale (Dollars)'], \
               aggfunc = [np.mean, max, min, np.sum]).\
                sort_values([('mean', 'Sale (Dollars)')], ascending = False).head(10)

,mean,max,min,sum
,Sale (Dollars),Sale (Dollars),Sale (Dollars),Sale (Dollars)
Zip Code,,,,
50061,429.840000,818.40,129.24,2579.04
50320,400.454243,36392.40,4.50,1306281.74
50540,346.300000,648.00,64.80,2077.80
50314,293.858874,13233.60,4.95,1336470.16
50830,271.840000,564.48,94.20,3262.08
52401,245.139833,2538.72,7.05,309121.33
50266,238.291152,32184.00,5.10,678653.20
52154,235.232683,846.24,35.30,9644.54


## Groupby City

In [257]:
pd.pivot_table(df, index = ['City', 'Store Number'], values = ['Sale (Dollars)'],\
               aggfunc = [np.mean, np.sum, np.std]).sort_values([('mean', 'Sale (Dollars)')], \
                                                                ascending = False).head(10)

,,mean,sum,std
,,Sale (Dollars),Sale (Dollars),Sale (Dollars)
City,Store Number,,,
DUBUQUE,9018,6435.000000,6435.00,NaN
WEST DES MOINES,3814,2234.074820,310536.40,4325.581435
CORALVILLE,4677,1782.670108,165788.32,3746.167251
SWISHER,9001,1204.865000,28916.76,2895.308462
WINDSOR HEIGHTS,3420,936.223395,402576.06,1810.604731
CEDAR RAPIDS,3385,855.724814,506589.09,1789.637834
LECLAIRE,9002,797.615510,39083.16,1005.067525
ANKENY,5144,787.377406,166924.01,1411.192674


In [258]:
pd.pivot_table(df, index = ['City', 'Store Number'], values = ['Sale (Dollars)'],\
               aggfunc = [np.mean, np.sum, np.std]).sort_values([('sum', 'Sale (Dollars)')], \
                                                                ascending = False).head(10)

mean            sum            std
                             Sale (Dollars) Sale (Dollars) Sale (Dollars)
City            Store Number                                             
DES MOINES      2633             491.080519     1220335.09    1581.816646
                4829             473.544726     1089152.87     795.966563
IOWA CITY       2512             302.767702      532265.62     720.721560
CEDAR RAPIDS    3385             855.724814      506589.09    1789.637834
WINDSOR HEIGHTS 3420             936.223395      402576.06    1810.604731
BETTENDORF      3952             344.255640      392451.43     506.144333
WEST DES MOINES 3814            2234.074820      310536.40    4325.581435
CEDAR RAPIDS    3773             253.965306      298663.20     305.082510
DAVENPORT       3354             750.236802      295593.30    1316.226035
CORALVILLE      2670             187.347078      282144.70     358.161785

# Explore the data

Perform some exploratory statistical analysis and make some plots, such as histograms of transaction totals, bottles sold, etc.

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

## Record your findings

Be sure to write out anything observations from your exploratory analysis.

# Mine the data
Now you are ready to compute the variables you will use for your regression from the data. For example, you may want to
compute total sales per store from Jan to March of 2015, mean price per bottle, etc. Refer to the readme for more ideas appropriate to your scenario.

Pandas is your friend for this task. Take a look at the operations [here](http://pandas.pydata.org/pandas-docs/stable/groupby.html) for ideas on how to make the best use of pandas and feel free to search for blog and Stack Overflow posts to help you group data by certain variables and compute sums, means, etc. You may find it useful to create a new data frame to house this summary data.

# Refine the data
Look for any statistical relationships, correlations, or other relevant properties of the dataset.

# Build your models

Using scikit-learn or statsmodels, build the necessary models for your scenario. Evaluate model fit.

In [6]:
from sklearn import linear_model


## Plot your results

Again make sure that you record any valuable information. For example, in the tax scenario, did you find the sales from the first three months of the year to be a good predictor of the total sales for the year? Plot the predictions versus the true values and discuss the successes and limitations of your models

# Present the Results

Present your conclusions and results. If you have more than one interesting model feel free to include more than one along with a discussion. Use your work in this notebook to prepare your write-up.